# Audio MNIST

#### Load Setup

In [1]:
from setup import setUp

# Initialize the setup class with the config file path
setup = setUp(cfg_filepath = 'config.yaml')

# Get the source path, meta data path, and destination path
source_path = setup.source_path
meta_data_path = setup.source_meta_path
destination_path = setup.destination_path
meta_data = setup.meta_data

In [ ]:
## In data processing class
# Add FFT function
# Create dict with features
# Add label column
# Normalize features
# Save to df and append somehow
# End of loop, save to csv
# Remove save_resampled_data

#### Data Processing

In [2]:
import glob
import os
from feature_creation import FeatureCreation
from data_processing import DataProcessing

source_path = 'C:/GitProjects/AudioMNIST\\data'
destination_path = 'C:/GitProjects/AudioMNIST\\preprocessed_data'

# Define the target sample rate
target_sr = 8000

# Initialize the DataProcessing class with the destination path, and target sample rate
DP = DataProcessing(destination_path, target_sr)

# Loop over the audio recordings in the source path
all_folders = len(next(os.walk(source_path))[1])+1
for i in range(1, 2): # instert all_folders when done testing
    src_temp = os.path.join(source_path, f"{i:02d}")
    filepath_filename = sorted(glob.glob(os.path.join(src_temp, "*.wav")))
    for file in filepath_filename:
        # read audio data
        fs, data = DP.read_audio(file)
        
        # resample audio data
        data = DP.resample_data(fs, data, target_sr)
        
        # zero padding audio data
        data = DP.zero_pad(data, target_sr)
        
        # save the resampled audio data
        # DP.save_resampled_data(file, data, target_sr)
